# Análisis exploratorio y modelado predictivo de riesgo de impago en préstamos

El dataset de préstamos financieros ('data/Loan_Default.csv') contiene información sobre características del solicitante, del préstamo y de la propiedad asociada. Vuestro objetivo es analizar el riesgo de impago (default) y construir un modelo de clasificación capaz de estimar si un préstamo puede estar en situación de impago.

La variable objetivo del problema es: "Status" o "estado_prestamo", donde:
- 1 préstamo en default (impago)
- 0 préstamo sano

El dataset cuenta con 34 características y 148,670 muestras.

Metadatos del dataset:
**Identificación y tiempo**
- ID: Identificador único del préstamo.
- year: Año en el que se concedió o se solicitó el préstamo.

**Características del préstamo**
- loan_limit: Indica si el préstamo está dentro del límite regulatorio (conforming) o lo supera (non-conforming).
- loan_type: Tipo de préstamo (por ejemplo, convencional, FHA, VA, etc.).
- loan_purpose: Finalidad del préstamo (compra, refinanciación, cash-out, etc.).
- loan_amount: Importe total del préstamo concedido.
- term: Duración del préstamo en meses (típicamente 180, 240, 360).
- interest_only: Indica si el préstamo permite pagar solo intereses durante un periodo inicial.
- lump_sum_payment: Indica si existe un pago único (balloon payment) al final.
- Neg_ammortization: Indica si el préstamo permite amortización negativa (la deuda puede aumentar).
- approv_in_adv: Si el préstamo fue aprobado previamente a la solicitud formal.

**Tipos de interés y costes**
- rate_of_interest: Tipo de interés aplicado al préstamo.
- Interest_rate_spread: Diferencia entre el tipo aplicado y un tipo de referencia.
- Upfront_charges: Cargos iniciales asociados al préstamo.

**Garantía y propiedad**
- property_value: Valor estimado de la propiedad.
- construction_type: Tipo de construcción (nueva, existente, etc.).
- occupancy_type: Uso de la propiedad (residencia principal, secundaria, inversión).
- Secured_by: Tipo de activo que garantiza el préstamo (normalmente propiedad inmobiliaria).
- total_units: Número de unidades en la propiedad (1 para vivienda unifamiliar).
- Security_Type: Tipo de garantía financiera asociada.

**Riesgo y solvencia**
- Credit_Worthiness: Evaluación general de la solvencia del solicitante.
- credit_type: Tipo de historial crediticio del solicitante principal.
- Credit_Score: Puntuación crediticia (similar a FICO score).
- co-applicant_credit_type: Tipo de crédito del co-solicitante, si existe.
- open_credit: Indica si el solicitante tiene líneas de crédito abiertas.
- dtir1: Debt-to-Income Ratio (DTI), ratio entre deuda e ingresos.

**Solicitante**
- Gender: Género del solicitante.
- age: Rango de edad del solicitante.
- income: Ingresos anuales del solicitante.
- submission_of_application: Canal de presentación de la solicitud (online, presencial, etc.).
- business_or_commercial: Indica si el préstamo es para fines empresariales o comerciales.

**Métricas financieras**
- LTV (Loan to Value): Ratio entre el importe del préstamo y el valor de la propiedad.
- Region: Región geográfica donde se ubica la propiedad.

**Estado del préstamo**
- Status: Estado final del préstamo (**TARGET**)
    - Valor: 0	Préstamo vigente / sano, sin impagos
    - Valor: 1	Préstamo en default / impago



# Objetivos de la actividad
- Comprender la estructura del dataset y su calidad (EDA)
- Realizar un pequeño estudio sobre los componentes principales (PAC)
- Preparar los datos para su uso en modelos de ML
- Seleccionar, entrenar y evaluar un clasificador binario.
- Interpretar los resultados del reporte de la clasificación y la curva AUC-ROC 

Referencias:
- https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc?hl=es-419
- https://es.wikipedia.org/wiki/Curva_ROC
  
  
# Código proporcionado
Os adjunto unas cuantas celdas con código para la carga del dataset. Se incluye un listasdo con las columnas en castellano,  si queréis renombrarlas, es opcional.

# Entrega 
- Entregad el notebook, con vuestro código y anotaciones sobre el mismo, pequeñas aclaraciones que consideréis oportunas sobre las variables.
- Se evaluará la coherencia del proceso, las decisiones tomadas sobre las características  y no se tendrá en cuenta la calidad del modelo ("su f1.score").





In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
# Librerías sklearn del modelo elegido


# 1. Carga de datos
# 2. EDA
# 3. PAC
# 4. Preprocessing
# 5. Model training
# 6. Model evaluation

#roc_auc = roc_auc_score(y_test, y_pred)
#print("ROC-AUC:", roc_auc)
#print(classification_report(y_test, X_test))

In [47]:
# 1. Carga de datos
df = pd.read_csv('data/Loan_Default.csv')

columnas_es = { #OPCIONAL
    "ID": "id_prestamo",
    "year": "anio",
    "loan_limit": "limite_prestamo",
    "Gender": "genero",
    "approv_in_adv": "aprobacion_previa",
    "loan_type": "tipo_prestamo",
    "loan_purpose": "proposito_prestamo",
    "Credit_Worthiness": "solvencia_crediticia",
    "open_credit": "credito_abierto",
    "business_or_commercial": "uso_comercial",
    "loan_amount": "importe_prestamo",
    "rate_of_interest": "tipo_interes",
    "Interest_rate_spread": "diferencial_interes",
    "Upfront_charges": "cargos_iniciales",
    "term": "plazo_meses",
    "Neg_ammortization": "amortizacion_negativa",
    "interest_only": "solo_intereses",
    "lump_sum_payment": "pago_unico_final",
    "property_value": "valor_propiedad",
    "construction_type": "tipo_construccion",
    "occupancy_type": "tipo_ocupacion",
    "Secured_by": "garantia",
    "total_units": "numero_unidades",
    "income": "ingresos",
    "credit_type": "tipo_credito",
    "Credit_Score": "puntuacion_crediticia",
    "co-applicant_credit_type": "tipo_credito_cotitular",
    "age": "edad",
    "submission_of_application": "canal_solicitud",
    "LTV": "ltv",
    "Region": "region",
    "Security_Type": "tipo_garantia",
    "Status": "estado_prestamo",
    "dtir1": "ratio_deuda_ingresos"
}

df.rename(columns=columnas_es, inplace=True) #OPCIONAL

In [48]:
# 2. EDA

In [49]:
df.shape

(148670, 34)

In [50]:
df.head() # multiples tipos de columnas

,id_prestamo,anio,limite_prestamo,genero,aprobacion_previa,tipo_prestamo,proposito_prestamo,solvencia_crediticia,credito_abierto,uso_comercial,...,tipo_credito,puntuacion_crediticia,tipo_credito_cotitular,edad,canal_solicitud,ltv,region,tipo_garantia,estado_prestamo,ratio_deuda_ingresos
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [51]:
df.describe() # Only numeric columns

,id_prestamo,anio,importe_prestamo,tipo_interes,diferencial_interes,cargos_iniciales,plazo_meses,valor_propiedad,ingresos,puntuacion_crediticia,ltv,estado_prestamo,ratio_deuda_ingresos
count,148670.000000,148670.0,1.486700e+05,112231.000000,112031.000000,109028.000000,148629.000000,1.335720e+05,139520.000000,148670.000000,133572.000000,148670.000000,124549.000000
mean,99224.500000,2019.0,3.311177e+05,4.045476,0.441656,3224.996127,335.136582,4.978935e+05,6957.338876,699.789103,72.746457,0.246445,37.732932
std,42917.476598,0.0,1.839093e+05,0.561391,0.513043,3251.121510,58.409084,3.599353e+05,6496.586382,115.875857,39.967603,0.430942,10.545435
min,24890.000000,2019.0,1.650000e+04,0.000000,-3.638000,0.000000,96.000000,8.000000e+03,0.000000,500.000000,0.967478,0.000000,5.000000
25%,62057.250000,2019.0,1.965000e+05,3.625000,0.076000,581.490000,360.000000,2.680000e+05,3720.000000,599.000000,60.474860,0.000000,31.000000
50%,99224.500000,2019.0,2.965000e+05,3.990000,0.390400,2596.450000,360.000000,4.180000e+05,5760.000000,699.000000,75.135870,0.000000,39.000000
75%,136391.750000,2019.0,4.365000e+05,4.375000,0.775400,4812.500000,360.000000,6.280000e+05,8520.000000,800.000000,86.184211,0.000000,45.000000
max,173559.000000,2019.0,3.576500e+06,8.000000,3.357000,60000.000000,360.000000,1.650800e+07,578580.000000,900.000000,7831.250000,1.000000,61.000000


In [52]:
df.isnull().sum() # Hay valores nulos

id_prestamo                   0
anio                          0
limite_prestamo            3344
genero                        0
aprobacion_previa           908
tipo_prestamo                 0
proposito_prestamo          134
solvencia_crediticia          0
credito_abierto               0
uso_comercial                 0
importe_prestamo              0
tipo_interes              36439
diferencial_interes       36639
cargos_iniciales          39642
plazo_meses                  41
amortizacion_negativa       121
solo_intereses                0
pago_unico_final              0
valor_propiedad           15098
tipo_construccion             0
tipo_ocupacion                0
garantia                      0
numero_unidades               0
ingresos                   9150
tipo_credito                  0
puntuacion_crediticia         0
tipo_credito_cotitular        0
edad                        200
canal_solicitud             200
ltv                       15098
region                        0
tipo_gar

In [53]:
# Observamos la distribución de la variable objetivo
df.groupby("estado_prestamo")["id_prestamo"].count()
# 
# 0	Préstamo vigente / sano, sin impago
# 1	Préstamo en default / impago

estado_prestamo
0    112031
1     36639
Name: id_prestamo, dtype: int64

In [54]:
# Vuestro turno...